# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 54 new papers today
          21 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/20 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2210.02463


extracting tarball to tmp_2210.02463...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.02465


extracting tarball to tmp_2210.02465...

...

extracting tarball to tmp_2210.03090...

 done.
Retrieving document from  https://arxiv.org/e-print/2210.03106


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2210.03090/natnotes.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2210.03106...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02820-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02820) | **The SDSS-Gaia View of the Color-Magnitude Relation for Blue  Horizontal-Branch Stars**  |
|| Fabrícia O. Barbosa, et al. -- incl., <mark>Guilherme Limberg</mark> |
|*Appeared on*| *2022-10-07*|
|*Comments*| *Accepted for publication in The Astrophysical Journal*|
|**Abstract**| We present an updated sample of blue horizontal-branch (BHB) stars selected from the photometric and spectroscopic data from Sloan Digital Sky Survey and its associated project Sloan Extension for Galactic Understanding and Exploration (SEGUE). With this data, we selected candidates for A-type stars in the color-color space and then a mixture modeling technique was implemented in order to distinguish between BHB and main-sequence/blue-straggler stars based on their surface gravity values ($\log \rm{g}$) estimated by the SEGUE Stellar Parameter Pipeline. Our robust approach allows us to attribute individual probabilities of each star truly being in the BHB stage. Hence, our method is advantageous in comparison to previous SEGUE BHB selections that adopted simple $\log \rm{g}$ cuts. We also revisit the color-magnitude relation for these stars and propose two calibrations, based on updated distances for Galactic globular clusters, to estimate absolute magnitudes with $(g-r)_0$ and $(u-r)_0$ colors. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02463-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02463) | **Machine learning classification of fast radio bursts: I. supervised  methods**  |
|| Jia-Wei Luo, Jia-Ming Zhu-Ge, <mark>Bing Zhang</mark> |
|*Appeared on*| *2022-10-07*|
|*Comments*| *12 pages, 13 figures, 2 tables, submitted to MNRAS*|
|**Abstract**| Observationally, the mysterious fast radio bursts (FRBs) are classified as repeating ones and apparently non-repeating ones. While repeating FRBs cannot be classified into the non-repeating group, it is unknown whether the apparently non-repeating FRBs are actually repeating FRBs whose repetitions are yet to be discovered, or whether they belong to another physically distinct type from the repeating ones. In a series of two papers, we attempt to disentangle this mystery with machine learning methods. In this first paper, we focus on an array of supervised machine learning methods. We train the machine learning algorithms with a fraction of the observed FRBs in the first CHIME/FRB catalog, telling them which ones are apparently non-repeating and which ones are repeating. We then let the trained models predict the repetitiveness of the rest of the FRB data with the observed parameters, and we compare the predictions with the observed repetitiveness. We find that the models can predict most FRBs correctly, hinting towards distinct mechanisms behind repeating and non-repeating FRBs. We also find that the two most important distinguishing factors between non-repeating and repeating FRBs are brightness temperature and rest-frame frequency bandwidth. By applying the trained models back to the entire first CHIME catalog, we further identify some potentially repeating FRBs currently reported as non-repeating. We recommend a list of these bursts as targets for future observing campaigns to search for repeated bursts in a combination with the results presented in Paper II using unsupervised machine learning methods. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02465-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02465) | **The Circumgalactic Medium of Extreme Emission Line Galaxies at z ~ 2:  Resolved Spectroscopy and Radiative Transfer Modeling of Spatially Extended  Lyman-alpha Emission in the KBSS-KCWI Survey**  |
|| Dawn K. Erb, et al. -- incl., <mark>Zhihui Li</mark> |
|*Appeared on*| *2022-10-07*|
|*Comments*| *41 pages, 17 figures. Submitted to ApJ*|
|**Abstract**| The resonantly scattered Lyman-$\alpha$ line illuminates the extended halos of neutral hydrogen in the circumgalactic medium of galaxies. We present integral field Keck Cosmic Web Imager observations of double-peaked, spatially extended Ly$\alpha$ emission in 12 relatively low-mass ($M_{\star} \sim10^9 \, M_{\odot}$) $z\sim2$ galaxies characterized by extreme nebular emission lines. Using individual spaxels and small bins as well as radially binned profiles of larger regions, we find that for most objects in the sample the Ly$\alpha$ blue-to-red peak ratio increases, the peak separation decreases, and the fraction of flux emerging at line center increases with radius. We use new radiative transfer simulations to model each galaxy with a clumpy, multiphase outflow with radially varying outflow velocity, and self-consistently apply the same velocity model to the low ionization interstellar absorption lines. These models reproduce the trends of peak ratio, peak separation and trough depth with radius, and broadly reconcile outflow velocities inferred from Ly$\alpha$ and absorption lines. The galaxies in our sample are well-described by a model in which neutral, outflowing clumps are embedded in a hotter, more highly ionized inter-clump medium (ICM), whose residual neutral content produces absorption at the systemic redshift. The peak ratio, peak separation and trough flux fraction are primarily governed by the line-of-sight component of the outflow velocity, the HI column density, and the residual neutral density in the ICM respectively. Azimuthal asymmetries in the line profile further suggest non-radial gas motions at large radii and variations in the HI column density in the outer halos. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03033-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03033) | **$JetCurry$ I. Reconstructing Three-Dimensional Jet Geometry from  Two-Dimensional Images**  |
|| Sailee M. Sawant, et al. -- incl., <mark>Kunyang Li</mark> |
|*Appeared on*| *2022-10-07*|
|*Comments*| *arXiv admin note: substantial text overlap with arXiv:1702.00534*|
|**Abstract**| We present a three-dimensional (3-D) visualization of jet geometry using numerical methods based on a Markov Chain Monte Carlo (MCMC) and limited memory Broyden-Fletcher-Goldfarb-Shanno (BFGS) optimized algorithm. Our aim is to visualize the 3-D geometry of an active galactic nucleus (AGN) jet using observations, which are inherently two-dimensional (2-D) images. Many AGN jets display complex structures that include hotspots and bends. The structure of these bends in the jet's frame may appear quite different than what we see in the sky frame, where it is transformed by our particular viewing geometry. The knowledge of the intrinsic structure will be helpful in understanding the appearance of the magnetic field and hence emission and particle acceleration processes over the length of the jet. We present the $JetCurry$ algorithm to visualize the jet's 3-D geometry from its 2-D image. We discuss the underlying geometrical framework and outline the method used to decompose the 2-D image. We report the results of our 3-D visualization of the jet of M87, using the test case of the knot D region. Our 3-D visualization is broadly consistent with the expected double helical magnetic field structure of the knot D region of the jet. We also discuss the next steps in the development of the $JetCurry$ algorithm. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03090-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03090) | **Dynamical masses of the primary Be star and the secondary sdB star in  the single-lined binary kappa Dra (B6 IIIe)**  |
|| R. Klement, et al. -- incl., <mark>C. L. Davies</mark> |
|*Appeared on*| *2022-10-07*|
|*Comments*| *Accepted to ApJ. Figure sets available on request*|
|**Abstract**| Because many classical Be stars may owe their nature to mass and angular-momentum transfer in a close binary, the present masses, temperatures, and radii of their components are of high interest for comparison to stellar evolution models. Kappa Dra is a 61.5-day single-lined binary with a B6 IIIe primary. With the CHARA Array instruments MIRC/MIRC-X and MYSTIC, we detected the secondary at (approximately photospheric) flux ratios of 1.49 +- 0.10% and 1.63 +- 0.09% in the H and K band, respectively. From a large and diverse optical spectroscopic database only the radial velocity curve of the Be star could be extracted. However, employing the parallaxes from Hipparcos and Gaia, which agree within their nominal 1-sigma errors, we could derive the total mass and found component masses of 3.65 +- 0.48 Msun and 0.426 +- 0.043 Msun for the Be star and the companion, respectively. Previous cross-correlation of the observed far-UV spectrum with sdO spectral model templates had not detected a companion belonging to the hot O-type subdwarf (sdO) population known from ~20 earlier-type Be stars. Guided by our full 3D orbital solution, we found a strong cross-correlation signal for a stripped subdwarf B-type companion (far-UV flux ratio of 2.3 +- 0.5%), enabling the first firm characterization of such a star, and making kappa Dra the first mid- to late-type Be star with a directly-observed subdwarf companion. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03106-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03106) | **Galaxy kinematics and mass estimates at z~1 from ionised gas and stars**  |
|| Hannah Übler, et al. -- incl., <mark>Natascha M. Förster Schreiber</mark>, <mark>Linda J. Tacconi</mark>, <mark>Ric Davies</mark>, <mark>Daizhong Liu</mark> |
|*Appeared on*| *2022-10-07*|
|*Comments*| *21 pages, 13 figures, 4 tables; submitted to MNRAS; comments are welcome*|
|**Abstract**| We compare ionised gas and stellar kinematics of 16 star-forming galaxies ($\log(M_\star/M_\odot)=9.7-11.2$, SFR=6-86 $M_\odot/yr$) at $z\sim1$ using near-infrared integral field spectroscopy (IFS) of H$\alpha$ emission from the KMOS$^{\rm 3D}$ survey and optical slit spectroscopy of stellar absorption and gas emission from the LEGA-C survey. H$\alpha$ is dynamically colder than stars, with higher disc rotation velocities (by ~45 per cent) and lower disc velocity dispersions (by a factor ~2). This is similar to trends observed in the local Universe. We find higher rotational support for H$\alpha$ relative to [OII], potentially explaining systematic offsets in kinematic scaling relations found in the literature. Regarding dynamical mass measurements, for six galaxies with cumulative mass profiles from Jeans Anisotropic Multi-Gaussian Expansion (JAM) models the H$\alpha$ dynamical mass models agree remarkably well out to ~10 kpc for all but one galaxy (average $\Delta M_{\rm dyn}(R_{e,\rm F814W})<0.1$ dex). Simpler dynamical mass estimates based on integrated stellar velocity dispersion are less accurate (standard deviation 0.24 dex). Differences in dynamical mass estimates are larger, for example, for galaxies with stronger misalignments of the H$\alpha$ kinematic major axis and the photometric position angle, highlighting the added value of IFS observations for dynamics studies. The good agreement between the JAM models and the dynamical models based on H$\alpha$ kinematics at $z\sim1$ corroborates the validity of dynamical mass measurements from H$\alpha$ IFS observations also for higher redshift rotating disc galaxies. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.02518-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.02518) | **Demographics of z ~ 6 Quasars in the Black Hole Mass-Luminosity Plane**  |
|| Jin Wu, et al. -- incl., <mark>Linhua Jiang</mark> |
|*Appeared on*| *2022-10-07*|
|*Comments*| *21 pages, 12 figures; accepted for publication in MNRAS*|
|**Abstract**| We study the demographics of z ~ 6 broad-line quasars in the black hole (BH) mass-luminosity plane using a sample of more than 100 quasars at 5.7 < z < 6.5. These quasars have well quantified selection functions and nearly one third of them also have virial BH masses estimated from near-IR spectroscopy. We use forward modeling of parameterized intrinsic distributions of BH masses and Eddington ratios, and account for the sample flux limits and measurement uncertainties of the BH masses and luminosities. We find significant differences between the intrinsic and observed distributions of the quantities due to measurement uncertainties and sample flux limits. There is also marginal evidence that the virial BH masses are susceptible to a positive luminosity-dependent bias (BH mass is overestimated when luminosity is above the average), and that the mean Eddington ratio increases with BH mass. Our models provide reliable constraints on the z ~ 6 black hole mass function at M_BH > 10^8.5 M_Sun, with a median 1-sigma uncertainty of ~0.5 dex in abundance. The intrinsic Eddington ratio distribution of M_BH > 10^8.5 M_Sun quasars can be approximated by a mass-dependent Schechter model, with a broad peak around log(L_bol/L_Edd}) ~ -0.9. We also find that, at 4.5 < z < 6, the number densities of more massive BHs tend to decline more rapidly with increasing redshift, contrary to the trend at 2.5 < z < 4.5 reported previously. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure ./bhmf-erdf-term.png</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2210.03056-b31b1b.svg)](https://arxiv.org/abs/arXiv:2210.03056) | **Lava World: Exoplanet Surfaces**  |
|| Marc-Antoine Fortin, Esteban Gazel, <mark>Lisa Kaltenegger</mark>, Megan E. Holycross |
|*Appeared on*| *2022-10-07*|
|*Comments*| *published in Monthly Notices of the Royal Astronomical Society*|
|**Abstract**| The recent first measurements of the reflection of the surface of a lava world provides an unprecedented opportunity to investigate different stages of rocky planet evolution. The spectral features of the surfaces of rocky lava world exoplanets give insights into their evolution, mantle composition and inner workings. However, no database exists yet that contains spectral reflectivity and emission of a wide range of potential exoplanet surface materials. Here we first synthesized 16 potential exoplanet surfaces, spanning a wide range of chemical compositions based on potential mantle material guided by the metallicity of different host stars. Then we measured their infrared reflection spectrum (2.5 - 28 {\mu}m, 350 - 4000 cm^{-1}), from which we can obtain their emission spectra and establish the link between the composition and a strong spectral feature at 8 {\mu}m, the Christiansen feature (CF). Our analysis suggests a new multi-component composition relationship with the CF, as well as a correlation with the silica content of the exoplanet mantle. We also report the mineralogies of our materials as possibilities for that of lava worlds. This database is a tool to aid in the interpretation of future spectra of lava worlds that will be collected by the James Webb Space Telescope and future missions |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

57  publications in the last 7 days.
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/html/2209.15110.md
	 _build/html/2209.14410.md
	 _build/html/2209.14396.md
	 _build/html/2209.13717.md
	 _build/html/2209.13336.md
	 _build/html/2209.12906.md
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers